# Interval conquest game:
#(Adriano Agnello, DARK-NBI)

In [ ]:
"""
I have found one analytical solution for N players:
 A places its bet at 1/(2(N-1)),
 B places its bet at 1-A .
 and the others except the last one share equal slices of B-A .
 The reason is the following: a NECESSARY condition for any optimal solution is that the last player must have the lesat possible estate;
 any deviation from the analytical solution above creates an edge for the last player, which at least one of the previous players could have seized.
 The numerical solution below exploits the same necessary condition, and it converges quite quickly to the analytical solution!
 NB1: Here I'm using a VERY quick-and dirty stochastic optimizer, but a gradient descent would be much quicker;
 the problem is that the gradient has several discontinuities and stuff gets stuck there.
 NB2: YES, it could be done much better and also prettier, but I have stuff to do.
""" 

In [ ]:
import numpy as np

In [ ]:
# N= number of players, eps=some small number that we need in the estate calculation to make things sensible
N=4;
eps=0.001;

In [ ]:
def payoff_last(others,safe=True):
    """
    payoff_last computes the "estate" reached by the last player
    once the positions of the other N-1 are given, for a reasonable choice of positions;
    I use a default "safe" mode because I want to work with a sorted array of positions, and for "magic numbers" safety;
    NB: the second extension in the array is only needed afterwards,
    when computing the average payoffs of the other players as the last one chooses at random
    (so that A can pick the best move and the other can each choose the best remaining strategy)
    """
    if safe:
        s=np.sort(others, axis=-1)
        NN=len(others)
    else:
        s=others
        NN=N-1
    # this puts the last player immediately before or after each of the other players;
    # NB about half of these steps are not needed for the payoff,
    # but they come in handy when averaging the payoffs of the other N-1 players over the possible choices of the last one
    tmp=np.zeros((2,2*NN));
    ti=0;
    while ti<NN-1:
        tmp[0,2*ti]= (s[ti]-s[ti-1])*0.5 ; tmp[0,2*ti+1]= (s[ti+1]-s[ti])*0.5 ;
        tmp[1,2*ti]= s[ti]-eps*(s[ti]-s[ti-1]) ; tmp[1,2*ti+1]= s[ti]+eps*(s[ti+1]-s[ti]) ;
        ti+=1
    tmp[0,0]=s[0]*(1.0-eps) ; tmp[0,2*(NN-1)+1]=(1.0-s[NN-1])*(1.0-eps);
    tmp[0,2*(NN-1)]=(s[NN-1]-s[NN-2])*0.5;
    tmp[1,0]=tmp[0,0] ; tmp[1,2*(NN-1)+1]=1.0-tmp[0,2*(NN-1)+1];
    tmp[1,2*(NN-1)]=s[NN-1]-(s[NN-1]-s[NN-2])*eps;
    return tmp
#
#
#
def adjust(others,safe=True):
    """
    adjust simply makes sure that the explored positions are between 0 and 1
    and reflects them accordingly, if needed; nothing fancy...
    I use a default "safe" mode because I don't like too many magic numbers around
    """
    if safe:
        NN=len(others)
    else:
        NN=N-1
    ir=0 ;
    while ir<NN:
        others[ir]=1.0-abs(1.0-others[ir])
        others[ir]=abs(others[ir])
        ir+=1
    others=np.sort(others) ;
    return others

In [ ]:
# draw a first choice for the positions and see if things are as they should...
# I print stuff in "lazy mode" here... 
firstpos=np.random.random(N-1);
firstpos=np.sort(firstpos)
print(firstpos);
payoff_last(firstpos)

In [ ]:
# now initialize a few thingies for the parameter exploration
Nmc=100000;
step=0.001/N;
pos=np.zeros((Nmc,N-1)) ;
loss=np.ones(Nmc) ;

In [ ]:
# compute the needed stuff for the first attempt at putting down positions
imc=0 ;
pos[0]=firstpos;
lastpays=payoff_last(pos[0]);
loss[0]=np.max(lastpays[0]);

In [ ]:
# just checking to see if var's are as they should be, before turning the crank...
print(imc,loss[0],pos[0])

In [ ]:
"""
Quick and Dirty stochastic optimizer: wonder around as long as the loss improves.
The "loss" is the maximum estate that the last player can claim.
"""

In [ ]:
# verbose, because I don't have patience and the thingy gets stuck quickly on an optimum
vb=True;
#
# now, a quick-and-dirty stochastic optimizer on the estate of the last player; let's see if it converges!
#
#
rejects=0;
maxrejects=20000*N; # this is because I've run it a few times and it stick almost immediately onto an optimum
# NB you'll need more patience with N>4 players,
# once you get to the lowest payoff of the last player there are A LOT of rejected moves;
#
imc=1;
while (imc<Nmc and rejects<maxrejects):
    cont=True ;
    rejects=0 ;
    while cont :
        postry=pos[imc-1]+ (-1.+2*np.random.random(N-1)) ;
        postry=adjust(postry) ;
        lastpays=payoff_last(postry) ;
        losstry=np.max(lastpays[0]) ;
        if (losstry<= loss[imc-1]): # accept if the "Estate" lost by the first N-1 players is smaller than before
            pos[imc]=postry ;
            loss[imc]=losstry ;
            if vb:
                print(imc,rejects,losstry, postry)
            imc+=1;
            cont=False ;
        if cont: rejects+=1 ;

In [ ]:
#for reference, this is where A and B should place their bets in my analytical solution:
#...and A is also the max payoff of the last player
1./(2*(N-1)) , 1.0-1./(2*(N-1))

In [ ]:
pos[np.argmin(loss)]

In [ ]:
# with four players, it's pretty neat!

In [ ]:
"""
As an alternative: Quick and Dirty penalized Metropolis-Hastings;
it does slightly better than the optimizer when N>4 , although it's still rather coarse.
"""

In [ ]:
# verbose, because I don't have patience and the thingy gets stuck quickly on an optimum
vb=False;
#
# let's see what happens with a penalised Metropolis-Hastings...
#
#
rejects=0;
maxrejects=20000*N; # this is because I've run it a few times and it stick almost immediately onto an optimum
# NB you'll need more patience with N>4 players,
# once you get to the lowest payoff of the last player there are A LOT of rejected moves;
#
imc=1;
while (imc<Nmc and rejects<maxrejects):
    cont=True ;
    rejects=0 ;
    while cont :
        postry=pos[imc-1]+ (-1.+2*np.random.random(N-1)) ;
        postry=adjust(postry) ;
        lastpays=payoff_last(postry) ;
        losstry=np.max(lastpays[0]) ;
        if (losstry<= loss[imc-1]): # if the "loss" in estate improves, accept
            pos[imc]=postry ;
            loss[imc]=losstry ;
            if vb:
                print(imc,rejects,losstry, postry) ;
            imc+=1;
            cont=False ;
        else: # accept with some small probability, given by the relative increase in estate loss
            tq=np.random.random() ;
            if (tq< np.exp(10*(loss[imc-1]-losstry)/loss[imc-1])):
                pos[imc]=postry ;
                loss[imc]=losstry ;
                if vb:
                    print(imc,rejects,losstry, postry) ;
                imc+=1 ;
                cont=False ;
        if cont: rejects+=1 ;

In [ ]:
#for reference, this is where A and B should place their bets in my analytical solution:
#...and A is also the max payoff of the last player
1./(2*(N-1)) , 1.0-1./(2*(N-1))

In [ ]:
pos[np.argmin(loss)]

In [ ]:
# maximum payoff ever of last player
(pos[np.argmin(loss)][1:]-pos[np.argmin(loss)][:-1])*0.5

In [ ]:
indices=np.argsort(loss)

In [ ]:
loss[indices[0:10]]

In [ ]:
0.5*(pos[indices[0:10]][:,1]+pos[indices[0:10]][:,0])

In [ ]:
pos[indices[0:10]][:,0]

In [ ]:
"""
with larger N (around 10), from the cells above you can _at_least_ clean up the chain of choices of A where its payoff 
(without accounting for the last payer)
is lower than the maximum estate that the last player can claim
"""

In [ ]:
diffs=0.5*(pos[indices[0:100]][:,1]+pos[indices[0:100]][:,0])-loss[indices[0:100]]

In [ ]:
pos[indices[:10][diffs[:10]>0]][:,0]

In [ ]:
#still, with N=4, it's all pretty neat!